Connect to Actuary team database to run SQL queries.

In [78]:
import pandas as pd
import pyodbc
connection_string = ("DRIVER={SQL Server};"
"SERVER=USDF11V0954;"
"DATABASE=Actuarial_AH;"
"Trust_Connection=yes;")

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

Find medical claims for members that had infertility claims that were paid after 2024.

In [ ]:
med = """
SELECT MEMBER_ID,
MEDICAL_CLAIM_ID,
BILLING_PROVIDER_NAME,
PAID_DATE,
MEDICAL_PAID_AMOUNT,
MEDICAL_ALLOWED_AMOUNT,
PROCEDURE_CODE,
PROCEDURE_DESC,
SERVICE_DATE,
EMPLOYER_NAME,
SERVICING_PROVIDER_STATE,
DIAGNOSIS_DESC_ICD10_1
FROM ACTUARIAL_AH.DBO.SN_Medical
-- WHERE PROCEDURE_CODE IN ('0253U', '0255U', '52402', '54500', '54505', '55300', '55530', '55535', '55550', '55870', '58140', '58145', '58146', '58321', '58322', '58323', '58340', '58345', '58350', '58545', '58546', '58555', '58559', '58660', '58662', '58670', '58672', '58673', '58740', '58752', '58760', '58770', '58800', '58805', '58920', '58970', '58974', '58976', '74440', '74740', '74742', '76830', '76831', '76856', '76857', '76870', '76872', '76948', '80415', '80426', '82397', '82670', '83001', '83002', '83498', '83520', '84144', '84146', '84402', '84403', '84443', '84830', '88182', '88248', '88261', '88262', '88263', '88273', '88280', '88283', '88285', '89250', '89251', '89253', '89254', '89255', '89257', '89258', '89259', '89260', '89261', '89264', '89268', '89272', '89280', '89281', '89290', '89291', '89300', '89310', '89320', '89321', '89322', '89325', '89329', '89330', '89331', '89335', '89337', '89342', '89343', '89344', '89346', '89352', '89353', '89354', '89356', '89398', 'J0725', 'J3355', 'S0122', 'S0126', 'S0128', 'S0132', 'S3655', 'S4011', 'S4013', 'S4014', 'S4015', 'S4016', 'S4017', 'S4018', 'S4020', 'S4021', 'S4022', 'S4023', 'S4025', 'S4026', 'S4027', 'S4028', 'S4030', 'S4031', 'S4035', 'S4037', 'S4040')
-- WHERE ICD_10_DX_CODE_1 IN ('N97.0', 'N97.1', 'N97.2', 'N97.9', 'Z31.81', 'Z31.82', 'Z31.83', 'Z31.84', 'N46.9','N46.8')
WHERE ICD10_SECTION IN ('Female infertility', 'Male infertility', 'Encounter for procreative management')
AND PAID_DATE >= '2024-01-01'
"""
ivf = pd.read_sql(med, conn)

Replace company data with mock claims dataset

In [47]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set random seed
np.random.seed(42)

# Step 1: Simulate unique member IDs
member_ids = np.random.choice(range(10000, 20000), size=3000, replace=False)

# Step 2: Set number of claims to generate
num_claims = 30000
claim_ids = np.arange(1, num_claims + 1)
claim_member_ids = np.random.choice(member_ids, size=num_claims, replace=True)

# Step 3: Generate random 2024 dates
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)
paid_dates = [random_date(start_date, end_date) for _ in range(num_claims)]
service_dates = [random_date(start_date, end_date) for _ in range(num_claims)]

# Step 4: Paid and allowed amounts
paid_amounts = np.round(np.random.exponential(scale=300, size=num_claims), 2)
allowed_amounts = paid_amounts + np.round(np.random.normal(loc=50, scale=25, size=num_claims), 2)

# Step 5: Sample fertility-related procedure codes and descriptions
procedure_codes = np.random.choice([
    '58140', '58660', '89250', '89255', 'S4011', 'S4021', '58970', '58974', '58322'
], size=num_claims)

procedure_descs = np.random.choice([
    'Laparoscopy for infertility',
    'Tuboplasty',
    'Sperm analysis',
    'Cryopreservation',
    'Assisted reproduction service',
    'Embryo culture'
], size=num_claims)

# Step 6: Provider, employer, and location info
billing_providers = np.random.choice(['Dr. Smith', 'Dr. Johnson', 'Dr. Patel', 'Dr. Wang'], size=num_claims)
provider_states = np.random.choice(['UT', 'CA', 'TX', 'NY', 'FL'], size=num_claims)

# Step 7: Diagnosis descriptions
diagnosis_descs = np.random.choice([
    'Female infertility, unspecified',
    'Male infertility, unspecified',
    'Encounter for fertility preservation counseling'
], size=num_claims)

# Step 8: Assemble DataFrame
ivf = pd.DataFrame({
    'MEMBER_ID': claim_member_ids,
    'MEDICAL_CLAIM_ID': claim_ids,
    'BILLING_PROVIDER_NAME': billing_providers,
    'PAID_DATE': paid_dates,
    'MEDICAL_PAID_AMOUNT': paid_amounts,
    'MEDICAL_ALLOWED_AMOUNT': allowed_amounts,
    'PROCEDURE_CODE': procedure_codes,
    'PROCEDURE_DESC': procedure_descs,
    'SERVICE_DATE': service_dates,
    'SERVICING_PROVIDER_STATE': provider_states,
    'DIAGNOSIS_DESC_ICD10_1': diagnosis_descs
})

ivf.head()

,MEMBER_ID,MEDICAL_CLAIM_ID,BILLING_PROVIDER_NAME,PAID_DATE,MEDICAL_PAID_AMOUNT,MEDICAL_ALLOWED_AMOUNT,PROCEDURE_CODE,PROCEDURE_DESC,SERVICE_DATE,SERVICING_PROVIDER_STATE,DIAGNOSIS_DESC_ICD10_1
0,18180,1,Dr. Smith,2024-04-23,176.17,209.83,89255,Sperm analysis,2024-11-02,CA,"Female infertility, unspecified"
1,10654,2,Dr. Johnson,2024-11-26,621.55,656.90,58660,Embryo culture,2024-04-09,TX,"Female infertility, unspecified"
2,15466,3,Dr. Smith,2024-02-01,401.98,488.04,89255,Cryopreservation,2024-02-16,NY,"Female infertility, unspecified"
3,18708,4,Dr. Wang,2024-06-20,133.68,181.83,58970,Laparoscopy for infertility,2024-09-07,NY,"Male infertility, unspecified"
4,16682,5,Dr. Smith,2024-05-04,386.30,415.87,89250,Cryopreservation,2024-10-26,NY,"Male infertility, unspecified"


Find pharmacy claims for common pharmaceuticals that are prescribed during IVF.

In [ ]:
rx = """
SELECT MEMBER_ID,
RX_CLAIM_ID,
PHARMACY_NAME,
DRUG_NAME_PREFERRED,
PAID_DATE,
RX_PAID_AMOUNT,
RX_ALLOWED_AMOUNT,
RX_SCRIPT_COUNT,
SERVICE_DATE
FROM ACTUARIAL_AH.DBO.SN_Rx
WHERE DRUG_NAME_PREFERRED IN ('Clomid', 'Serophene', 'Femara', 'Menopur', 'Follistim', 'Gonal-F', 'Ovidrel', 'Pregnyl', 'Parlodel', 'Provera')
AND PAID_DATE >= '2024-01-01'
"""
rx = pd.read_sql(rx, conn)

Replace company dataset with mock claims data

In [41]:
# Set seed for reproducibility
np.random.seed(42)

# Step 1: Create a pool of member IDs
member_ids = np.random.choice(range(10000, 20000), size=3000, replace=False)

# Step 2: Set number of Rx claims to generate
num_rx_claims = 20000
rx_claim_ids = np.arange(1, num_rx_claims + 1)
rx_member_ids = np.random.choice(member_ids, size=num_rx_claims, replace=True)

# Step 3: Generate dates from 2024
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)
rx_paid_dates = [random_date(start_date, end_date) for _ in range(num_rx_claims)]
rx_service_dates = [random_date(start_date, end_date) for _ in range(num_rx_claims)]

# Step 4: Fertility-related drugs and pharmacy names
fertility_drugs = [
    'Clomid', 'Serophene', 'Femara', 'Menopur', 'Follistim',
    'Gonal-F', 'Ovidrel', 'Pregnyl', 'Parlodel', 'Provera'
]
drug_names = np.random.choice(fertility_drugs, size=num_rx_claims)

pharmacy_names = np.random.choice(
    ['CVS', 'Walgreens', 'Rite Aid', 'Kroger Pharmacy', 'Walmart Pharmacy'],
    size=num_rx_claims
)

# Step 5: Payment details
rx_paid_amounts = np.round(np.random.exponential(scale=250, size=num_rx_claims), 2)
rx_allowed_amounts = rx_paid_amounts + np.round(np.random.normal(loc=20, scale=10, size=num_rx_claims), 2)
rx_script_counts = np.random.choice([1, 2, 3], size=num_rx_claims)

# Step 6: Create DataFrame
rx = pd.DataFrame({
    'MEMBER_ID': rx_member_ids,
    'RX_CLAIM_ID': rx_claim_ids,
    'PHARMACY_NAME': pharmacy_names,
    'DRUG_NAME_PREFERRED': drug_names,
    'PAID_DATE': rx_paid_dates,
    'RX_PAID_AMOUNT': rx_paid_amounts,
    'RX_ALLOWED_AMOUNT': rx_allowed_amounts,
    'RX_SCRIPT_COUNT': rx_script_counts,
    'SERVICE_DATE': rx_service_dates
})

rx.head()

,MEMBER_ID,RX_CLAIM_ID,PHARMACY_NAME,DRUG_NAME_PREFERRED,PAID_DATE,RX_PAID_AMOUNT,RX_ALLOWED_AMOUNT,RX_SCRIPT_COUNT,SERVICE_DATE
0,18180,1,Kroger Pharmacy,Clomid,2024-08-14,60.83,84.28,3,2024-07-27
1,10654,2,Kroger Pharmacy,Pregnyl,2024-06-19,67.17,81.45,3,2024-04-28
2,15466,3,Walgreens,Clomid,2024-09-12,35.76,57.23,2,2024-11-18
3,18708,4,Rite Aid,Menopur,2024-07-30,325.35,348.69,1,2024-03-28
4,16682,5,Rite Aid,Serophene,2024-07-13,211.50,228.07,3,2024-12-29


In [ ]:
member = """
SELECT 
    m.MEMBER_ID,
    m.EMPLOYER_NAME,
    m.HOME_STATE,
    CASE 
        WHEN m.MEMBER_RELATIONSHIP = 0 THEN 'Subscriber'
        WHEN m.MEMBER_RELATIONSHIP = 1 THEN 'Spouse'
        WHEN m.MEMBER_RELATIONSHIP = 2 THEN'Dependent'
        WHEN m.MEMBER_RELATIONSHIP = 3 THEN 'Domestic Partner'
        WHEN m.MEMBER_RELATIONSHIP = 4 THEN 'Unknown'
        ELSE 'Unknown' 
    END AS MEMBER_RELATIONSHIP,
    CASE 
        WHEN m.GENDER = 0 THEN 'Female'
        WHEN m.GENDER = 1 THEN 'Male'
        WHEN m.GENDER = 2 THEN 'Unknown'
        ELSE 'Unknown' 
    END AS GENDER,
    CASE 
        WHEN m.MEDICAL_ENROLLMENT_STATUS = 0 THEN 'Unknown'
        WHEN m.MEDICAL_ENROLLMENT_STATUS = 1 THEN 'Not Eligible, Not Enrolled'
        WHEN m.MEDICAL_ENROLLMENT_STATUS = 2 THEN 'Eligible, Not Enrolled'
        WHEN m.MEDICAL_ENROLLMENT_STATUS = 3 THEN 'Eligible, Enrolled'
        ELSE 'Unknown' 
    END AS MEMBER_ENROLLMENT_STATUS,
    CONVERT(DATE, m.BIRTH_YEAR_MO + '-01') AS BirthDate, 
    DATEDIFF(YEAR, CONVERT(DATE, m.BIRTH_YEAR_MO + '-01'), GETDATE()) - 
    CASE 
        WHEN MONTH(CONVERT(DATE, m.BIRTH_YEAR_MO + '-01')) > MONTH(GETDATE()) 
             OR (MONTH(CONVERT(DATE, m.BIRTH_YEAR_MO + '-01')) = MONTH(GETDATE()) 
                 AND DAY(CONVERT(DATE, m.BIRTH_YEAR_MO + '-01')) > DAY(GETDATE())) 
        THEN 1 
        ELSE 0 
    END AS Age
FROM 
    ACTUARIAL_AH.DBO.SN_MEMBER m
WHERE
    m.MONTH_KEY = '12-01-24'
    and m.MEDICAL_ENROLLMENT_STATUS = 3


"""
member = pd.read_sql(member, conn)

Replace company dataset with mock claims data

In [42]:
# Set seed for reproducibility
np.random.seed(42)

# Step 1: Simulate member IDs
num_members = 5000
member_ids = np.random.choice(range(10000, 20000), size=num_members, replace=False)

# Step 2: Simulate attributes
employers = np.random.choice(['Acme Corp', 'Globex Inc', 'Initech', 'Umbrella Health'], size=num_members)
home_states = np.random.choice(['UT', 'CA', 'TX', 'NY', 'FL', 'WA', 'IL', 'CO', 'PA', 'NC'], size=num_members)
relationship_codes = np.random.choice([0, 1, 2, 3, 4], size=num_members)
gender_codes = np.random.choice([0, 1, 2], size=num_members)
enrollment_status_codes = np.full(num_members, 3)  # All members are enrolled

# Step 3: Simulate birth years and months
birth_years = np.random.choice(range(1946, 2012), size=num_members)
birth_months = np.random.choice(range(1, 13), size=num_members)
birth_dates = [datetime(year, month, 1) for year, month in zip(birth_years, birth_months)]

# Step 4: Calculate age as of today
today = datetime.today()
ages = [today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day)) for dob in birth_dates]

# Step 5: Map relationship, gender, and enrollment codes
relationship_map = {0: 'Subscriber', 1: 'Spouse', 2: 'Dependent', 3: 'Domestic Partner', 4: 'Unknown'}
gender_map = {0: 'Female', 1: 'Male', 2: 'Unknown'}
enrollment_map = {
    0: 'Unknown',
    1: 'Not Eligible, Not Enrolled',
    2: 'Eligible, Not Enrolled',
    3: 'Eligible, Enrolled'
}

# Step 6: Build DataFrame
member = pd.DataFrame({
    'MEMBER_ID': member_ids,
    'EMPLOYER_NAME': employers,
    'HOME_STATE': home_states,
    'MEMBER_RELATIONSHIP': [relationship_map[r] for r in relationship_codes],
    'GENDER': [gender_map[g] for g in gender_codes],
    'MEMBER_ENROLLMENT_STATUS': [enrollment_map[e] for e in enrollment_status_codes],
    'BirthDate': [dob.strftime('%Y-%m-%d') for dob in birth_dates],
    'Age': ages
})

member.head()

,MEMBER_ID,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,MEMBER_ENROLLMENT_STATUS,BirthDate,Age
0,16252,Globex Inc,WA,Domestic Partner,Male,"Eligible, Enrolled",1949-03-01,76
1,14684,Umbrella Health,NY,Subscriber,Male,"Eligible, Enrolled",1998-10-01,26
2,11731,Umbrella Health,PA,Dependent,Male,"Eligible, Enrolled",1972-04-01,53
3,14742,Initech,FL,Dependent,Male,"Eligible, Enrolled",1966-04-01,59
4,14521,Umbrella Health,NC,Spouse,Unknown,"Eligible, Enrolled",2005-01-01,20


In [48]:
#all medical claims related to fertility that have been queried above
ivf.head()

,MEMBER_ID,MEDICAL_CLAIM_ID,BILLING_PROVIDER_NAME,PAID_DATE,MEDICAL_PAID_AMOUNT,MEDICAL_ALLOWED_AMOUNT,PROCEDURE_CODE,PROCEDURE_DESC,SERVICE_DATE,SERVICING_PROVIDER_STATE,DIAGNOSIS_DESC_ICD10_1
0,18180,1,Dr. Smith,2024-04-23,176.17,209.83,89255,Sperm analysis,2024-11-02,CA,"Female infertility, unspecified"
1,10654,2,Dr. Johnson,2024-11-26,621.55,656.90,58660,Embryo culture,2024-04-09,TX,"Female infertility, unspecified"
2,15466,3,Dr. Smith,2024-02-01,401.98,488.04,89255,Cryopreservation,2024-02-16,NY,"Female infertility, unspecified"
3,18708,4,Dr. Wang,2024-06-20,133.68,181.83,58970,Laparoscopy for infertility,2024-09-07,NY,"Male infertility, unspecified"
4,16682,5,Dr. Smith,2024-05-04,386.30,415.87,89250,Cryopreservation,2024-10-26,NY,"Male infertility, unspecified"


In [44]:
rx.head()

,MEMBER_ID,RX_CLAIM_ID,PHARMACY_NAME,DRUG_NAME_PREFERRED,PAID_DATE,RX_PAID_AMOUNT,RX_ALLOWED_AMOUNT,RX_SCRIPT_COUNT,SERVICE_DATE
0,18180,1,Kroger Pharmacy,Clomid,2024-08-14,60.83,84.28,3,2024-07-27
1,10654,2,Kroger Pharmacy,Pregnyl,2024-06-19,67.17,81.45,3,2024-04-28
2,15466,3,Walgreens,Clomid,2024-09-12,35.76,57.23,2,2024-11-18
3,18708,4,Rite Aid,Menopur,2024-07-30,325.35,348.69,1,2024-03-28
4,16682,5,Rite Aid,Serophene,2024-07-13,211.50,228.07,3,2024-12-29


In [49]:
members = ivf.MEMBER_ID.unique()
members.shape

(3000,)

In [50]:
ivf_members = ivf.merge(member, on='MEMBER_ID', how= 'left')
ivf_members

,MEMBER_ID,MEDICAL_CLAIM_ID,BILLING_PROVIDER_NAME,PAID_DATE,MEDICAL_PAID_AMOUNT,MEDICAL_ALLOWED_AMOUNT,PROCEDURE_CODE,PROCEDURE_DESC,SERVICE_DATE,SERVICING_PROVIDER_STATE,DIAGNOSIS_DESC_ICD10_1,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,MEMBER_ENROLLMENT_STATUS,BirthDate,Age
0,18180,1,Dr. Smith,2024-04-23,176.17,209.83,89255,Sperm analysis,2024-11-02,CA,"Female infertility, unspecified",Umbrella Health,NY,Dependent,Male,"Eligible, Enrolled",1974-09-01,50
1,10654,2,Dr. Johnson,2024-11-26,621.55,656.90,58660,Embryo culture,2024-04-09,TX,"Female infertility, unspecified",Umbrella Health,PA,Subscriber,Male,"Eligible, Enrolled",2011-08-01,13
2,15466,3,Dr. Smith,2024-02-01,401.98,488.04,89255,Cryopreservation,2024-02-16,NY,"Female infertility, unspecified",Umbrella Health,TX,Subscriber,Unknown,"Eligible, Enrolled",1949-09-01,75
3,18708,4,Dr. Wang,2024-06-20,133.68,181.83,58970,Laparoscopy for infertility,2024-09-07,NY,"Male infertility, unspecified",Umbrella Health,FL,Domestic Partner,Female,"Eligible, Enrolled",1950-10-01,74
4,16682,5,Dr. Smith,2024-05-04,386.30,415.87,89250,Cryopreservation,2024-10-26,NY,"Male infertility, unspecified",Initech,CA,Spouse,Male,"Eligible, Enrolled",1969-09-01,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,15271,29996,Dr. Johnson,2024-01-19,103.29,139.03,89250,Cryopreservation,2024-12-07,FL,Encounter for fertility preservation counseling,Globex Inc,TX,Subscriber,Male,"Eligible, Enrolled",1960-09-01,64
29996,10252,29997,Dr. Smith,2024-12-20,49.10,106.21,58970,Assisted reproduction service,2024-01-13,UT,"Male infertility, unspecified",Acme Corp,CO,Spouse,Male,"Eligible, Enrolled",2000-04-01,25
29997,11281,29998,Dr. Smith,2024-05-03,102.75,145.49,58140,Tuboplasty,2024-05-16,CA,"Female infertility, unspecified",Acme Corp,TX,Unknown,Male,"Eligible, Enrolled",1973-05-01,52
29998,18281,29999,Dr. Wang,2024-10-17,892.48,924.42,58660,Sperm analysis,2024-11-16,TX,"Female infertility, unspecified",Initech,WA,Unknown,Unknown,"Eligible, Enrolled",1993-04-01,32


In [51]:
ivf_members.GENDER.value_counts()

GENDER
Female     10292
Unknown    10265
Male        9443
Name: count, dtype: int64

In [52]:
ivf_males = ivf_members[ivf_members['GENDER'] == 'Male']
ivf_males

,MEMBER_ID,MEDICAL_CLAIM_ID,BILLING_PROVIDER_NAME,PAID_DATE,MEDICAL_PAID_AMOUNT,MEDICAL_ALLOWED_AMOUNT,PROCEDURE_CODE,PROCEDURE_DESC,SERVICE_DATE,SERVICING_PROVIDER_STATE,DIAGNOSIS_DESC_ICD10_1,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,MEMBER_ENROLLMENT_STATUS,BirthDate,Age
0,18180,1,Dr. Smith,2024-04-23,176.17,209.83,89255,Sperm analysis,2024-11-02,CA,"Female infertility, unspecified",Umbrella Health,NY,Dependent,Male,"Eligible, Enrolled",1974-09-01,50
1,10654,2,Dr. Johnson,2024-11-26,621.55,656.90,58660,Embryo culture,2024-04-09,TX,"Female infertility, unspecified",Umbrella Health,PA,Subscriber,Male,"Eligible, Enrolled",2011-08-01,13
4,16682,5,Dr. Smith,2024-05-04,386.30,415.87,89250,Cryopreservation,2024-10-26,NY,"Male infertility, unspecified",Initech,CA,Spouse,Male,"Eligible, Enrolled",1969-09-01,55
7,17438,8,Dr. Patel,2024-03-19,163.36,249.69,58660,Sperm analysis,2024-04-22,NY,Encounter for fertility preservation counseling,Acme Corp,NY,Spouse,Male,"Eligible, Enrolled",1973-07-01,52
13,12943,14,Dr. Patel,2024-02-14,459.64,480.81,58970,Embryo culture,2024-06-08,FL,"Male infertility, unspecified",Umbrella Health,UT,Domestic Partner,Male,"Eligible, Enrolled",1979-06-01,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,10713,29994,Dr. Patel,2024-09-14,355.09,403.47,89255,Sperm analysis,2024-04-16,CA,Encounter for fertility preservation counseling,Globex Inc,CO,Dependent,Male,"Eligible, Enrolled",1966-09-01,58
29994,13685,29995,Dr. Patel,2024-04-21,836.30,881.32,89250,Laparoscopy for infertility,2024-03-30,UT,Encounter for fertility preservation counseling,Umbrella Health,NY,Spouse,Male,"Eligible, Enrolled",1963-09-01,61
29995,15271,29996,Dr. Johnson,2024-01-19,103.29,139.03,89250,Cryopreservation,2024-12-07,FL,Encounter for fertility preservation counseling,Globex Inc,TX,Subscriber,Male,"Eligible, Enrolled",1960-09-01,64
29996,10252,29997,Dr. Smith,2024-12-20,49.10,106.21,58970,Assisted reproduction service,2024-01-13,UT,"Male infertility, unspecified",Acme Corp,CO,Spouse,Male,"Eligible, Enrolled",2000-04-01,25


In [53]:
ivf_males.shape

(9443, 18)

In [54]:
ivf1_males = ivf_males.pivot_table(
    index='MEMBER_ID',
    values=['MEDICAL_ALLOWED_AMOUNT', 'MEDICAL_PAID_AMOUNT', 'SERVICE_DATE'],
    aggfunc={
    'MEDICAL_ALLOWED_AMOUNT': 'sum',
    'MEDICAL_PAID_AMOUNT': 'sum',
    'SERVICE_DATE': ['min', 'max']
    }
)

ivf1_males.columns = ['Med_Allowed', 'Med_Paid', 'Max_Service_Date', 'Min_Service_Date']


In [55]:
male_merge = ivf1_males.merge(member, on= 'MEMBER_ID')

In [56]:
male_merge

,MEMBER_ID,Med_Allowed,Med_Paid,Max_Service_Date,Min_Service_Date,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,MEMBER_ENROLLMENT_STATUS,BirthDate,Age
0,10010,3769.27,3256.57,2024-11-29,2024-01-22,Globex Inc,NY,Subscriber,Male,"Eligible, Enrolled",1987-01-01,38
1,10012,2789.20,2302.38,2024-12-25,2024-01-26,Initech,PA,Subscriber,Male,"Eligible, Enrolled",1962-02-01,63
2,10029,4404.82,3840.44,2024-11-25,2024-01-18,Umbrella Health,WA,Dependent,Male,"Eligible, Enrolled",1974-04-01,51
3,10030,4070.70,3384.77,2024-12-31,2024-01-04,Umbrella Health,PA,Spouse,Male,"Eligible, Enrolled",1986-08-01,38
4,10037,2596.25,2279.05,2024-12-21,2024-01-04,Initech,CA,Domestic Partner,Male,"Eligible, Enrolled",1955-07-01,70
...,...,...,...,...,...,...,...,...,...,...,...,...
951,19924,3252.15,2811.97,2024-11-25,2024-01-18,Acme Corp,FL,Domestic Partner,Male,"Eligible, Enrolled",1980-02-01,45
952,19962,2203.01,1935.82,2024-06-09,2024-01-06,Umbrella Health,CO,Spouse,Male,"Eligible, Enrolled",2005-02-01,20
953,19965,3160.35,2679.42,2024-12-31,2024-01-10,Acme Corp,PA,Subscriber,Male,"Eligible, Enrolled",1947-09-01,77
954,19972,4681.00,3893.04,2024-12-22,2024-01-22,Globex Inc,CO,Domestic Partner,Male,"Eligible, Enrolled",1992-12-01,32


In [57]:
male_merge.describe()

,MEMBER_ID,Med_Allowed,Med_Paid,Max_Service_Date,Min_Service_Date,Age
count,956.000000,956.000000,956.000000,956,956,956.000000
mean,14937.130753,3462.898776,2968.779874,2024-11-24 14:27:36.903765760,2024-02-07 11:05:46.443514880,46.547071
min,10010.000000,197.530000,103.430000,2024-03-09 00:00:00,2024-01-01 00:00:00,13.000000
25%,12369.250000,2294.535000,1920.637500,2024-11-11 00:00:00,2024-01-10 00:00:00,30.000000
50%,14953.000000,3327.085000,2846.820000,2024-12-07 00:00:00,2024-01-25 12:00:00,47.500000
75%,17444.000000,4399.062500,3820.980000,2024-12-22 00:00:00,2024-02-23 00:00:00,63.000000
max,19975.000000,8980.250000,8302.770000,2024-12-31 00:00:00,2024-09-22 00:00:00,79.000000
std,2884.349691,1507.425376,1381.409619,NaN,NaN,19.023185


In [58]:
ivf1_pivot = ivf.pivot_table(
    index='MEMBER_ID',
    values=['MEDICAL_ALLOWED_AMOUNT', 'MEDICAL_PAID_AMOUNT', 'SERVICE_DATE'],
    aggfunc={
    'MEDICAL_ALLOWED_AMOUNT': 'sum',
    'MEDICAL_PAID_AMOUNT': 'sum',
    'SERVICE_DATE': ['min', 'max']
    }
)

ivf_codes = ['58970', '58974', '58322']
for code in ivf_codes:
    ivf1_pivot[code] = (ivf['PROCEDURE_CODE'] == code).groupby(ivf['MEMBER_ID']).sum()
    
ivf1_pivot.columns = ['Med_Allowed', 'Med_Paid', 'Max_Service_Date', 'Min_Service_Date'] + ivf_codes 
ivf1_pivot

,Med_Allowed,Med_Paid,Max_Service_Date,Min_Service_Date,58970,58974,58322
MEMBER_ID,,,,,,,
10000,2536.31,2056.84,2024-11-04,2024-01-16,0,2,1
10003,2585.31,2250.71,2024-10-05,2024-04-20,1,2,0
10008,1768.26,1472.27,2024-08-04,2024-02-26,0,0,0
10010,3769.27,3256.57,2024-11-29,2024-01-22,2,1,0
10012,2789.20,2302.38,2024-12-25,2024-01-26,3,1,2
...,...,...,...,...,...,...,...
19975,4845.54,4079.61,2024-11-13,2024-02-25,1,1,3
19977,4202.22,3714.36,2024-11-25,2024-01-13,1,2,0
19989,3770.80,3203.59,2024-10-31,2024-03-06,2,2,0


In [59]:
ivf1_pivot.describe()

,Med_Allowed,Med_Paid,Max_Service_Date,Min_Service_Date,58970,58974,58322
count,3000.000000,3000.000000,3000,3000,3000.000000,3000.000000,3000.00000
mean,3515.518440,3017.524463,2024-11-25 00:03:21.600000256,2024-02-06 10:06:43.200000256,1.118000,1.119000,1.08500
min,193.260000,103.430000,2024-02-17 00:00:00,2024-01-01 00:00:00,0.000000,0.000000,0.00000
25%,2385.232500,1986.390000,2024-11-11 00:00:00,2024-01-11 00:00:00,0.000000,0.000000,0.00000
50%,3378.030000,2882.920000,2024-12-07 00:00:00,2024-01-26 00:00:00,1.000000,1.000000,1.00000
75%,4414.077500,3855.750000,2024-12-21 00:00:00,2024-02-21 00:00:00,2.000000,2.000000,2.00000
max,10748.860000,9801.880000,2024-12-31 00:00:00,2024-09-22 00:00:00,6.000000,7.000000,5.00000
std,1514.025125,1390.615039,NaN,NaN,1.050608,1.066404,1.04824


In [60]:
#filter dataset by only those members that have had an egg retrieval or embryo transfer
ivf_codes = ['58970', '58974', '58322', '58976', '58321']
ivf_members = ivf[ivf['PROCEDURE_CODE'].isin(ivf_codes)]
uniq_members = ivf_members['MEMBER_ID'].unique()
ivf1 = ivf[ivf['MEMBER_ID'].isin(uniq_members)]
rx1 = rx[rx['MEMBER_ID'].isin(uniq_members)]
ivf1

,MEMBER_ID,MEDICAL_CLAIM_ID,BILLING_PROVIDER_NAME,PAID_DATE,MEDICAL_PAID_AMOUNT,MEDICAL_ALLOWED_AMOUNT,PROCEDURE_CODE,PROCEDURE_DESC,SERVICE_DATE,SERVICING_PROVIDER_STATE,DIAGNOSIS_DESC_ICD10_1
0,18180,1,Dr. Smith,2024-04-23,176.17,209.83,89255,Sperm analysis,2024-11-02,CA,"Female infertility, unspecified"
1,10654,2,Dr. Johnson,2024-11-26,621.55,656.90,58660,Embryo culture,2024-04-09,TX,"Female infertility, unspecified"
2,15466,3,Dr. Smith,2024-02-01,401.98,488.04,89255,Cryopreservation,2024-02-16,NY,"Female infertility, unspecified"
3,18708,4,Dr. Wang,2024-06-20,133.68,181.83,58970,Laparoscopy for infertility,2024-09-07,NY,"Male infertility, unspecified"
4,16682,5,Dr. Smith,2024-05-04,386.30,415.87,89250,Cryopreservation,2024-10-26,NY,"Male infertility, unspecified"
...,...,...,...,...,...,...,...,...,...,...,...
29995,15271,29996,Dr. Johnson,2024-01-19,103.29,139.03,89250,Cryopreservation,2024-12-07,FL,Encounter for fertility preservation counseling
29996,10252,29997,Dr. Smith,2024-12-20,49.10,106.21,58970,Assisted reproduction service,2024-01-13,UT,"Male infertility, unspecified"
29997,11281,29998,Dr. Smith,2024-05-03,102.75,145.49,58140,Tuboplasty,2024-05-16,CA,"Female infertility, unspecified"
29998,18281,29999,Dr. Wang,2024-10-17,892.48,924.42,58660,Sperm analysis,2024-11-16,TX,"Female infertility, unspecified"


In [62]:
rx_pivot = rx1.pivot_table(
    index='MEMBER_ID',
    values=['RX_PAID_AMOUNT', 'RX_ALLOWED_AMOUNT', 'RX_SCRIPT_COUNT', 'SERVICE_DATE'],
    aggfunc={
    'RX_PAID_AMOUNT': 'sum',
    'RX_ALLOWED_AMOUNT': 'sum',
    'RX_SCRIPT_COUNT': 'sum',
    'SERVICE_DATE': ['min', 'max']
    }
)
rx_pivot.columns = ['RX_ALLOWED', 'RX_PAID', 'Scripts', 'Max_Rx_Date', 'Min_Rx_Date']
rx_ivf = rx_pivot.reset_index()
rx_ivf

,MEMBER_ID,RX_ALLOWED,RX_PAID,Scripts,Max_Rx_Date,Min_Rx_Date
0,10000,899.10,811.06,11,2024-12-12,2024-01-05
1,10003,597.75,503.13,9,2024-12-15,2024-01-05
2,10010,2603.29,2361.41,22,2024-11-21,2024-03-02
3,10012,1709.12,1507.38,19,2024-12-14,2024-05-01
4,10014,267.53,219.51,5,2024-07-31,2024-03-22
...,...,...,...,...,...,...
2882,19975,2288.29,2115.93,17,2024-12-05,2024-01-04
2883,19977,1153.82,1022.02,8,2024-08-16,2024-02-29
2884,19989,2368.13,2166.71,21,2024-11-15,2024-01-27
2885,19993,4232.89,3967.75,19,2024-12-17,2024-01-06


In [63]:
rx_ivf.describe() # add descriptive stats before and after filtering for 1 -3 egg retrievals

,MEMBER_ID,RX_ALLOWED,RX_PAID,Scripts,Max_Rx_Date,Min_Rx_Date
count,2887.000000,2887.000000,2887.000000,2887.000000,2887,2887
mean,14931.316592,1817.872854,1682.494489,13.526152,2024-11-08 20:39:29.241427200,2024-02-21 17:59:52.518184704
min,10000.000000,63.650000,17.840000,1.000000,2024-01-03 00:00:00,2024-01-01 00:00:00
25%,12415.000000,1129.265000,1022.460000,9.000000,2024-10-18 00:00:00,2024-01-15 00:00:00
50%,14903.000000,1685.270000,1563.720000,13.000000,2024-11-26 00:00:00,2024-02-06 00:00:00
75%,17471.500000,2386.870000,2221.030000,17.000000,2024-12-17 00:00:00,2024-03-13 00:00:00
max,19998.000000,6950.380000,6722.530000,35.000000,2024-12-31 00:00:00,2024-12-28 00:00:00
std,2893.787898,934.448457,897.843867,5.479369,NaN,NaN


In [64]:
ivf_pivot = ivf1.pivot_table(
    index='MEMBER_ID',
    values=['MEDICAL_ALLOWED_AMOUNT', 'MEDICAL_PAID_AMOUNT', 'SERVICE_DATE'],
    aggfunc={
    'MEDICAL_ALLOWED_AMOUNT': 'sum',
    'MEDICAL_PAID_AMOUNT': 'sum',
    'SERVICE_DATE': ['min', 'max']
    }
)

for code in ivf_codes:
    ivf_pivot[code] = (ivf1['PROCEDURE_CODE'] == code).groupby(ivf1['MEMBER_ID']).sum()
    
ivf_pivot.columns = ['Med_Allowed', 'Med_Paid', 'Max_Service_Date', 'Min_Service_Date'] + ivf_codes
ivf_pivot

,Med_Allowed,Med_Paid,Max_Service_Date,Min_Service_Date,58970,58974,58322,58976,58321
MEMBER_ID,,,,,,,,,
10000,2536.31,2056.84,2024-11-04,2024-01-16,0,2,1,0,0
10003,2585.31,2250.71,2024-10-05,2024-04-20,1,2,0,0,0
10010,3769.27,3256.57,2024-11-29,2024-01-22,2,1,0,0,0
10012,2789.20,2302.38,2024-12-25,2024-01-26,3,1,2,0,0
10014,2286.78,1919.12,2024-06-02,2024-01-08,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
19975,4845.54,4079.61,2024-11-13,2024-02-25,1,1,3,0,0
19977,4202.22,3714.36,2024-11-25,2024-01-13,1,2,0,0,0
19989,3770.80,3203.59,2024-10-31,2024-03-06,2,2,0,0,0


In [65]:
ivf_df = ivf_pivot.reset_index()
ivf_df

,MEMBER_ID,Med_Allowed,Med_Paid,Max_Service_Date,Min_Service_Date,58970,58974,58322,58976,58321
0,10000,2536.31,2056.84,2024-11-04,2024-01-16,0,2,1,0,0
1,10003,2585.31,2250.71,2024-10-05,2024-04-20,1,2,0,0,0
2,10010,3769.27,3256.57,2024-11-29,2024-01-22,2,1,0,0,0
3,10012,2789.20,2302.38,2024-12-25,2024-01-26,3,1,2,0,0
4,10014,2286.78,1919.12,2024-06-02,2024-01-08,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2882,19975,4845.54,4079.61,2024-11-13,2024-02-25,1,1,3,0,0
2883,19977,4202.22,3714.36,2024-11-25,2024-01-13,1,2,0,0,0
2884,19989,3770.80,3203.59,2024-10-31,2024-03-06,2,2,0,0,0
2885,19993,4763.74,3856.71,2024-12-25,2024-02-07,2,1,2,0,0


In [66]:
ivf_df.sort_values(by='Med_Allowed')
ivf_df

,MEMBER_ID,Med_Allowed,Med_Paid,Max_Service_Date,Min_Service_Date,58970,58974,58322,58976,58321
0,10000,2536.31,2056.84,2024-11-04,2024-01-16,0,2,1,0,0
1,10003,2585.31,2250.71,2024-10-05,2024-04-20,1,2,0,0,0
2,10010,3769.27,3256.57,2024-11-29,2024-01-22,2,1,0,0,0
3,10012,2789.20,2302.38,2024-12-25,2024-01-26,3,1,2,0,0
4,10014,2286.78,1919.12,2024-06-02,2024-01-08,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2882,19975,4845.54,4079.61,2024-11-13,2024-02-25,1,1,3,0,0
2883,19977,4202.22,3714.36,2024-11-25,2024-01-13,1,2,0,0,0
2884,19989,3770.80,3203.59,2024-10-31,2024-03-06,2,2,0,0,0
2885,19993,4763.74,3856.71,2024-12-25,2024-02-07,2,1,2,0,0


In [67]:
ivf_df.rename(columns={
    '58970': 'Egg_Retrieval',
    '58974': 'Embryo_Transfer',
    '58322': 'IUI',
    '58321': 'Cervical Insemination',
    '58976': 'IVF procedure'}, inplace=True)
ivf_df

,MEMBER_ID,Med_Allowed,Med_Paid,Max_Service_Date,Min_Service_Date,Egg_Retrieval,Embryo_Transfer,IUI,IVF procedure,Cervical Insemination
0,10000,2536.31,2056.84,2024-11-04,2024-01-16,0,2,1,0,0
1,10003,2585.31,2250.71,2024-10-05,2024-04-20,1,2,0,0,0
2,10010,3769.27,3256.57,2024-11-29,2024-01-22,2,1,0,0,0
3,10012,2789.20,2302.38,2024-12-25,2024-01-26,3,1,2,0,0
4,10014,2286.78,1919.12,2024-06-02,2024-01-08,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2882,19975,4845.54,4079.61,2024-11-13,2024-02-25,1,1,3,0,0
2883,19977,4202.22,3714.36,2024-11-25,2024-01-13,1,2,0,0,0
2884,19989,3770.80,3203.59,2024-10-31,2024-03-06,2,2,0,0,0
2885,19993,4763.74,3856.71,2024-12-25,2024-02-07,2,1,2,0,0


In [68]:
ivf_df.columns

Index(['MEMBER_ID', 'Med_Allowed', 'Med_Paid', 'Max_Service_Date',
       'Min_Service_Date', 'Egg_Retrieval', 'Embryo_Transfer', 'IUI',
       'IVF procedure', 'Cervical Insemination'],
      dtype='object')

In [69]:
#reorder columns
ivf_df = ivf_df[['MEMBER_ID', 'Med_Allowed', 'Med_Paid','Min_Service_Date', 'Max_Service_Date', 'Egg_Retrieval', 'Embryo_Transfer', 'IUI', 'Cervical Insemination', 'IVF procedure' ]]
#ivf_df = ivf_df.merge(rx_ivf, on='MEMBER_ID', how='left')
ivf_df

,MEMBER_ID,Med_Allowed,Med_Paid,Min_Service_Date,Max_Service_Date,Egg_Retrieval,Embryo_Transfer,IUI,Cervical Insemination,IVF procedure
0,10000,2536.31,2056.84,2024-01-16,2024-11-04,0,2,1,0,0
1,10003,2585.31,2250.71,2024-04-20,2024-10-05,1,2,0,0,0
2,10010,3769.27,3256.57,2024-01-22,2024-11-29,2,1,0,0,0
3,10012,2789.20,2302.38,2024-01-26,2024-12-25,3,1,2,0,0
4,10014,2286.78,1919.12,2024-01-08,2024-06-02,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2882,19975,4845.54,4079.61,2024-02-25,2024-11-13,1,1,3,0,0
2883,19977,4202.22,3714.36,2024-01-13,2024-11-25,1,2,0,0,0
2884,19989,3770.80,3203.59,2024-03-06,2024-10-31,2,2,0,0,0
2885,19993,4763.74,3856.71,2024-02-07,2024-12-25,2,1,2,0,0


In [70]:
# Calculate the number of days between Min and Max Service Dates
ivf_df['Med_Days_Between'] = (ivf_df['Max_Service_Date'] - ivf_df['Min_Service_Date']).dt.days
#vf_df['Rx_Days_Between'] = (ivf_df['Max_Rx_Date'] - ivf_df['Min_Rx_Date']).dt.days
ivf_df

/var/folders/qy/w1846_ms6g97cvq51vdv9qrr0000gn/T/ipykernel_14897/2546370204.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ivf_df['Med_Days_Between'] = (ivf_df['Max_Service_Date'] - ivf_df['Min_Service_Date']).dt.days


,MEMBER_ID,Med_Allowed,Med_Paid,Min_Service_Date,Max_Service_Date,Egg_Retrieval,Embryo_Transfer,IUI,Cervical Insemination,IVF procedure,Med_Days_Between
0,10000,2536.31,2056.84,2024-01-16,2024-11-04,0,2,1,0,0,293
1,10003,2585.31,2250.71,2024-04-20,2024-10-05,1,2,0,0,0,168
2,10010,3769.27,3256.57,2024-01-22,2024-11-29,2,1,0,0,0,312
3,10012,2789.20,2302.38,2024-01-26,2024-12-25,3,1,2,0,0,334
4,10014,2286.78,1919.12,2024-01-08,2024-06-02,0,1,0,0,0,146
...,...,...,...,...,...,...,...,...,...,...,...
2882,19975,4845.54,4079.61,2024-02-25,2024-11-13,1,1,3,0,0,262
2883,19977,4202.22,3714.36,2024-01-13,2024-11-25,1,2,0,0,0,317
2884,19989,3770.80,3203.59,2024-03-06,2024-10-31,2,2,0,0,0,239
2885,19993,4763.74,3856.71,2024-02-07,2024-12-25,2,1,2,0,0,322


In [71]:
#merge member table onto ivf. 645 members
ivf_merge = ivf_df.merge(member, on='MEMBER_ID', how= 'inner')
ivf_merge

,MEMBER_ID,Med_Allowed,Med_Paid,Min_Service_Date,Max_Service_Date,Egg_Retrieval,Embryo_Transfer,IUI,Cervical Insemination,IVF procedure,Med_Days_Between,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,MEMBER_ENROLLMENT_STATUS,BirthDate,Age
0,10000,2536.31,2056.84,2024-01-16,2024-11-04,0,2,1,0,0,293,Acme Corp,NY,Subscriber,Female,"Eligible, Enrolled",1987-08-01,37
1,10003,2585.31,2250.71,2024-04-20,2024-10-05,1,2,0,0,0,168,Globex Inc,CO,Subscriber,Female,"Eligible, Enrolled",1947-12-01,77
2,10010,3769.27,3256.57,2024-01-22,2024-11-29,2,1,0,0,0,312,Globex Inc,NY,Subscriber,Male,"Eligible, Enrolled",1987-01-01,38
3,10012,2789.20,2302.38,2024-01-26,2024-12-25,3,1,2,0,0,334,Initech,PA,Subscriber,Male,"Eligible, Enrolled",1962-02-01,63
4,10014,2286.78,1919.12,2024-01-08,2024-06-02,0,1,0,0,0,146,Globex Inc,CA,Unknown,Unknown,"Eligible, Enrolled",2005-07-01,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2882,19975,4845.54,4079.61,2024-02-25,2024-11-13,1,1,3,0,0,262,Initech,CA,Dependent,Male,"Eligible, Enrolled",2007-03-01,18
2883,19977,4202.22,3714.36,2024-01-13,2024-11-25,1,2,0,0,0,317,Initech,CO,Unknown,Female,"Eligible, Enrolled",1997-09-01,27
2884,19989,3770.80,3203.59,2024-03-06,2024-10-31,2,2,0,0,0,239,Acme Corp,CO,Dependent,Female,"Eligible, Enrolled",1957-09-01,67
2885,19993,4763.74,3856.71,2024-02-07,2024-12-25,2,1,2,0,0,322,Acme Corp,CA,Unknown,Unknown,"Eligible, Enrolled",1982-11-01,42


In [72]:
ivf_merge['MEMBER_ENROLLMENT_STATUS'].value_counts()

MEMBER_ENROLLMENT_STATUS
Eligible, Enrolled    2887
Name: count, dtype: int64

In [73]:
ivf_merge.GENDER.value_counts()

GENDER
Unknown    990
Female     975
Male       922
Name: count, dtype: int64

In [74]:
ivf_merge.drop(columns=['BirthDate', 'MEMBER_ENROLLMENT_STATUS'], inplace=True)

In [75]:
ivf_merge.describe()

,MEMBER_ID,Med_Allowed,Med_Paid,Min_Service_Date,Max_Service_Date,Egg_Retrieval,Embryo_Transfer,IUI,Cervical Insemination,IVF procedure,Med_Days_Between,Age
count,2887.000000,2887.000000,2887.000000,2887,2887,2887.000000,2887.000000,2887.000000,2887.0,2887.0,2887.000000,2887.000000
mean,14931.316592,3571.158272,3065.772276,2024-02-05 20:12:33.169379584,2024-11-26 03:07:32.649809408,1.161760,1.162799,1.127468,0.0,0.0,294.288188,45.995151
min,10000.000000,266.310000,153.630000,2024-01-01 00:00:00,2024-03-06 00:00:00,0.000000,0.000000,0.000000,0.0,0.0,9.000000,13.000000
25%,12415.000000,2452.595000,2034.195000,2024-01-11 00:00:00,2024-11-12 00:00:00,0.000000,0.000000,0.000000,0.0,0.0,269.000000,30.000000
50%,14903.000000,3436.290000,2918.800000,2024-01-25 00:00:00,2024-12-08 00:00:00,1.000000,1.000000,1.000000,0.0,0.0,308.000000,46.000000
75%,17471.500000,4469.975000,3888.170000,2024-02-20 00:00:00,2024-12-21 00:00:00,2.000000,2.000000,2.000000,0.0,0.0,332.000000,63.000000
max,19998.000000,10748.860000,9801.880000,2024-09-22 00:00:00,2024-12-31 00:00:00,6.000000,7.000000,5.000000,0.0,0.0,365.000000,79.000000
std,2893.787898,1499.508202,1380.142172,NaN,NaN,1.046967,1.063390,1.045913,0.0,0.0,51.307702,19.019685


In [76]:
ivf_iui = ivf_merge[(ivf_merge['Egg_Retrieval'] > 0) | (ivf_merge['Embryo_Transfer'] > 0) | (ivf_merge['IUI'] > 0)]
ivf_iui = ivf_iui.merge(rx_ivf, on='MEMBER_ID', how='left')

ivf_iui

,MEMBER_ID,Med_Allowed,Med_Paid,Min_Service_Date,Max_Service_Date,Egg_Retrieval,Embryo_Transfer,IUI,Cervical Insemination,IVF procedure,...,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,RX_ALLOWED,RX_PAID,Scripts,Max_Rx_Date,Min_Rx_Date
0,10000,2536.31,2056.84,2024-01-16,2024-11-04,0,2,1,0,0,...,Acme Corp,NY,Subscriber,Female,37,899.10,811.06,11,2024-12-12,2024-01-05
1,10003,2585.31,2250.71,2024-04-20,2024-10-05,1,2,0,0,0,...,Globex Inc,CO,Subscriber,Female,77,597.75,503.13,9,2024-12-15,2024-01-05
2,10010,3769.27,3256.57,2024-01-22,2024-11-29,2,1,0,0,0,...,Globex Inc,NY,Subscriber,Male,38,2603.29,2361.41,22,2024-11-21,2024-03-02
3,10012,2789.20,2302.38,2024-01-26,2024-12-25,3,1,2,0,0,...,Initech,PA,Subscriber,Male,63,1709.12,1507.38,19,2024-12-14,2024-05-01
4,10014,2286.78,1919.12,2024-01-08,2024-06-02,0,1,0,0,0,...,Globex Inc,CA,Unknown,Unknown,20,267.53,219.51,5,2024-07-31,2024-03-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2882,19975,4845.54,4079.61,2024-02-25,2024-11-13,1,1,3,0,0,...,Initech,CA,Dependent,Male,18,2288.29,2115.93,17,2024-12-05,2024-01-04
2883,19977,4202.22,3714.36,2024-01-13,2024-11-25,1,2,0,0,0,...,Initech,CO,Unknown,Female,27,1153.82,1022.02,8,2024-08-16,2024-02-29
2884,19989,3770.80,3203.59,2024-03-06,2024-10-31,2,2,0,0,0,...,Acme Corp,CO,Dependent,Female,67,2368.13,2166.71,21,2024-11-15,2024-01-27
2885,19993,4763.74,3856.71,2024-02-07,2024-12-25,2,1,2,0,0,...,Acme Corp,CA,Unknown,Unknown,42,4232.89,3967.75,19,2024-12-17,2024-01-06


In [77]:
ivf_iui.to_csv('ivf_iui_data_2024.csv', index=False)

In [78]:
ivf_iui.describe()

,MEMBER_ID,Med_Allowed,Med_Paid,Min_Service_Date,Max_Service_Date,Egg_Retrieval,Embryo_Transfer,IUI,Cervical Insemination,IVF procedure,Med_Days_Between,Age,RX_ALLOWED,RX_PAID,Scripts,Max_Rx_Date,Min_Rx_Date
count,2887.000000,2887.000000,2887.000000,2887,2887,2887.000000,2887.000000,2887.000000,2887.0,2887.0,2887.000000,2887.000000,2887.000000,2887.000000,2887.000000,2887,2887
mean,14931.316592,3571.158272,3065.772276,2024-02-05 20:12:33.169379584,2024-11-26 03:07:32.649809408,1.161760,1.162799,1.127468,0.0,0.0,294.288188,45.995151,1817.872854,1682.494489,13.526152,2024-11-08 20:39:29.241427200,2024-02-21 17:59:52.518184704
min,10000.000000,266.310000,153.630000,2024-01-01 00:00:00,2024-03-06 00:00:00,0.000000,0.000000,0.000000,0.0,0.0,9.000000,13.000000,63.650000,17.840000,1.000000,2024-01-03 00:00:00,2024-01-01 00:00:00
25%,12415.000000,2452.595000,2034.195000,2024-01-11 00:00:00,2024-11-12 00:00:00,0.000000,0.000000,0.000000,0.0,0.0,269.000000,30.000000,1129.265000,1022.460000,9.000000,2024-10-18 00:00:00,2024-01-15 00:00:00
50%,14903.000000,3436.290000,2918.800000,2024-01-25 00:00:00,2024-12-08 00:00:00,1.000000,1.000000,1.000000,0.0,0.0,308.000000,46.000000,1685.270000,1563.720000,13.000000,2024-11-26 00:00:00,2024-02-06 00:00:00
75%,17471.500000,4469.975000,3888.170000,2024-02-20 00:00:00,2024-12-21 00:00:00,2.000000,2.000000,2.000000,0.0,0.0,332.000000,63.000000,2386.870000,2221.030000,17.000000,2024-12-17 00:00:00,2024-03-13 00:00:00
max,19998.000000,10748.860000,9801.880000,2024-09-22 00:00:00,2024-12-31 00:00:00,6.000000,7.000000,5.000000,0.0,0.0,365.000000,79.000000,6950.380000,6722.530000,35.000000,2024-12-31 00:00:00,2024-12-28 00:00:00
std,2893.787898,1499.508202,1380.142172,NaN,NaN,1.046967,1.063390,1.045913,0.0,0.0,51.307702,19.019685,934.448457,897.843867,5.479369,NaN,NaN


In [79]:
#only include members with at least one egg retrieval procedure
ivf_merge = ivf_merge[(ivf_merge['Egg_Retrieval'] > 0) & (ivf_merge['Egg_Retrieval'] < 4)]
ivf_merge

,MEMBER_ID,Med_Allowed,Med_Paid,Min_Service_Date,Max_Service_Date,Egg_Retrieval,Embryo_Transfer,IUI,Cervical Insemination,IVF procedure,Med_Days_Between,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age
1,10003,2585.31,2250.71,2024-04-20,2024-10-05,1,2,0,0,0,168,Globex Inc,CO,Subscriber,Female,77
2,10010,3769.27,3256.57,2024-01-22,2024-11-29,2,1,0,0,0,312,Globex Inc,NY,Subscriber,Male,38
3,10012,2789.20,2302.38,2024-01-26,2024-12-25,3,1,2,0,0,334,Initech,PA,Subscriber,Male,63
5,10017,2954.40,2527.48,2024-02-01,2024-11-25,1,1,0,0,0,298,Umbrella Health,FL,Subscriber,Female,15
6,10019,3346.69,2659.78,2024-02-17,2024-11-19,2,1,2,0,0,276,Acme Corp,NY,Spouse,Unknown,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2882,19975,4845.54,4079.61,2024-02-25,2024-11-13,1,1,3,0,0,262,Initech,CA,Dependent,Male,18
2883,19977,4202.22,3714.36,2024-01-13,2024-11-25,1,2,0,0,0,317,Initech,CO,Unknown,Female,27
2884,19989,3770.80,3203.59,2024-03-06,2024-10-31,2,2,0,0,0,239,Acme Corp,CO,Dependent,Female,67
2885,19993,4763.74,3856.71,2024-02-07,2024-12-25,2,1,2,0,0,322,Acme Corp,CA,Unknown,Unknown,42


In [80]:
ivf_rx_filtered = ivf_merge.merge(rx_ivf, on='MEMBER_ID', how='left')
ivf_rx_filtered

,MEMBER_ID,Med_Allowed,Med_Paid,Min_Service_Date,Max_Service_Date,Egg_Retrieval,Embryo_Transfer,IUI,Cervical Insemination,IVF procedure,...,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,RX_ALLOWED,RX_PAID,Scripts,Max_Rx_Date,Min_Rx_Date
0,10003,2585.31,2250.71,2024-04-20,2024-10-05,1,2,0,0,0,...,Globex Inc,CO,Subscriber,Female,77,597.75,503.13,9,2024-12-15,2024-01-05
1,10010,3769.27,3256.57,2024-01-22,2024-11-29,2,1,0,0,0,...,Globex Inc,NY,Subscriber,Male,38,2603.29,2361.41,22,2024-11-21,2024-03-02
2,10012,2789.20,2302.38,2024-01-26,2024-12-25,3,1,2,0,0,...,Initech,PA,Subscriber,Male,63,1709.12,1507.38,19,2024-12-14,2024-05-01
3,10017,2954.40,2527.48,2024-02-01,2024-11-25,1,1,0,0,0,...,Umbrella Health,FL,Subscriber,Female,15,1547.96,1450.49,12,2024-11-17,2024-03-01
4,10019,3346.69,2659.78,2024-02-17,2024-11-19,2,1,2,0,0,...,Acme Corp,NY,Spouse,Unknown,66,1357.46,1272.47,10,2024-09-27,2024-02-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1944,19975,4845.54,4079.61,2024-02-25,2024-11-13,1,1,3,0,0,...,Initech,CA,Dependent,Male,18,2288.29,2115.93,17,2024-12-05,2024-01-04
1945,19977,4202.22,3714.36,2024-01-13,2024-11-25,1,2,0,0,0,...,Initech,CO,Unknown,Female,27,1153.82,1022.02,8,2024-08-16,2024-02-29
1946,19989,3770.80,3203.59,2024-03-06,2024-10-31,2,2,0,0,0,...,Acme Corp,CO,Dependent,Female,67,2368.13,2166.71,21,2024-11-15,2024-01-27
1947,19993,4763.74,3856.71,2024-02-07,2024-12-25,2,1,2,0,0,...,Acme Corp,CA,Unknown,Unknown,42,4232.89,3967.75,19,2024-12-17,2024-01-06


In [81]:
ivf_rx_filtered.GENDER.value_counts()

GENDER
Female     693
Unknown    647
Male       609
Name: count, dtype: int64

In [82]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [83]:
ivf_rx_filtered.describe()

,MEMBER_ID,Med_Allowed,Med_Paid,Min_Service_Date,Max_Service_Date,Egg_Retrieval,Embryo_Transfer,IUI,Cervical Insemination,IVF procedure,Med_Days_Between,Age,RX_ALLOWED,RX_PAID,Scripts,Max_Rx_Date,Min_Rx_Date
count,1949.00,1949.00,1949.00,1949,1949,1949.00,1949.00,1949.00,1949.00,1949.00,1949.00,1949.00,1949.00,1949.00,1949.00,1949,1949
mean,14951.46,3709.96,3186.65,2024-02-04 00:49:30.138532352,2024-11-27 03:15:47.562852608,1.55,1.12,1.10,0.00,0.00,297.10,46.10,1869.50,1730.18,13.92,2024-11-11 01:14:37.373011968,2024-02-21 06:10:09.543355392
min,10003.00,450.50,246.54,2024-01-01 00:00:00,2024-03-06 00:00:00,1.00,0.00,0.00,0.00,0.00,49.00,13.00,89.42,39.46,1.00,2024-01-03 00:00:00,2024-01-01 00:00:00
25%,12436.00,2612.66,2162.22,2024-01-10 00:00:00,2024-11-13 00:00:00,1.00,0.00,0.00,0.00,0.00,272.00,30.00,1174.49,1055.65,10.00,2024-10-21 00:00:00,2024-01-15 00:00:00
50%,14914.00,3594.23,3064.91,2024-01-24 00:00:00,2024-12-07 00:00:00,1.00,1.00,1.00,0.00,0.00,308.00,46.00,1740.43,1604.10,14.00,2024-11-27 00:00:00,2024-02-06 00:00:00
75%,17500.00,4585.76,4010.23,2024-02-17 00:00:00,2024-12-21 00:00:00,2.00,2.00,2.00,0.00,0.00,333.00,63.00,2462.16,2297.16,18.00,2024-12-18 00:00:00,2024-03-13 00:00:00
max,19998.00,10748.86,9801.88,2024-09-09 00:00:00,2024-12-31 00:00:00,3.00,7.00,5.00,0.00,0.00,365.00,79.00,6950.38,6722.53,35.00,2024-12-31 00:00:00,2024-12-04 00:00:00
std,2902.45,1488.91,1377.68,NaN,NaN,0.71,1.07,1.07,0.00,0.00,47.94,19.01,937.42,901.23,5.45,NaN,NaN


In [84]:
ivf_data = ivf_rx_filtered.to_csv('ivf_data_2024.csv', index=False) #export full dataset of 333 members to csv

In [85]:
ivf_rx_describe = ivf_rx_filtered.describe()
ivf_rx_describecsv = ivf_rx_describe.to_csv('ivf_rx_describe.csv', index=False) #export med and rx descriptive stats

In [90]:
#use original medical claims dataset and filter by 418 members set
uniq_members = ivf1['MEMBER_ID'].unique()
ivf2 = ivf[ivf['MEMBER_ID'].isin(uniq_members)]
ivf2

,MEMBER_ID,MEDICAL_CLAIM_ID,BILLING_PROVIDER_NAME,PAID_DATE,MEDICAL_PAID_AMOUNT,MEDICAL_ALLOWED_AMOUNT,PROCEDURE_CODE,PROCEDURE_DESC,SERVICE_DATE,SERVICING_PROVIDER_STATE,DIAGNOSIS_DESC_ICD10_1
0,18180,1,Dr. Smith,2024-04-23,176.17,209.83,89255,Sperm analysis,2024-11-02,CA,"Female infertility, unspecified"
1,10654,2,Dr. Johnson,2024-11-26,621.55,656.90,58660,Embryo culture,2024-04-09,TX,"Female infertility, unspecified"
2,15466,3,Dr. Smith,2024-02-01,401.98,488.04,89255,Cryopreservation,2024-02-16,NY,"Female infertility, unspecified"
3,18708,4,Dr. Wang,2024-06-20,133.68,181.83,58970,Laparoscopy for infertility,2024-09-07,NY,"Male infertility, unspecified"
4,16682,5,Dr. Smith,2024-05-04,386.30,415.87,89250,Cryopreservation,2024-10-26,NY,"Male infertility, unspecified"
...,...,...,...,...,...,...,...,...,...,...,...
29995,15271,29996,Dr. Johnson,2024-01-19,103.29,139.03,89250,Cryopreservation,2024-12-07,FL,Encounter for fertility preservation counseling
29996,10252,29997,Dr. Smith,2024-12-20,49.10,106.21,58970,Assisted reproduction service,2024-01-13,UT,"Male infertility, unspecified"
29997,11281,29998,Dr. Smith,2024-05-03,102.75,145.49,58140,Tuboplasty,2024-05-16,CA,"Female infertility, unspecified"
29998,18281,29999,Dr. Wang,2024-10-17,892.48,924.42,58660,Sperm analysis,2024-11-16,TX,"Female infertility, unspecified"


In [91]:
rx1
uniq_members = ivf1['MEMBER_ID'].unique()
rx2 = rx1[rx1['MEMBER_ID'].isin(uniq_members)]
rx2

,MEMBER_ID,RX_CLAIM_ID,PHARMACY_NAME,DRUG_NAME_PREFERRED,PAID_DATE,RX_PAID_AMOUNT,RX_ALLOWED_AMOUNT,RX_SCRIPT_COUNT,SERVICE_DATE
0,18180,1,Kroger Pharmacy,Clomid,2024-08-14,60.83,84.28,3,2024-07-27
1,10654,2,Kroger Pharmacy,Pregnyl,2024-06-19,67.17,81.45,3,2024-04-28
2,15466,3,Walgreens,Clomid,2024-09-12,35.76,57.23,2,2024-11-18
3,18708,4,Rite Aid,Menopur,2024-07-30,325.35,348.69,1,2024-03-28
4,16682,5,Rite Aid,Serophene,2024-07-13,211.50,228.07,3,2024-12-29
...,...,...,...,...,...,...,...,...,...
19995,18088,19996,Kroger Pharmacy,Provera,2024-01-26,113.83,138.38,2,2024-02-18
19996,11740,19997,Kroger Pharmacy,Femara,2024-12-16,23.02,24.23,1,2024-03-11
19997,18440,19998,Kroger Pharmacy,Ovidrel,2024-10-02,107.33,137.40,3,2024-04-07
19998,16727,19999,Walmart Pharmacy,Serophene,2024-02-11,342.76,378.32,2,2024-07-16


In [92]:
# Step 1: Aggregate by MEMBER_ID to get total medical paid amount per member
med_agg = ivf2.groupby('MEMBER_ID').agg(
    EMPLOYER_NAME=('EMPLOYER_NAME', 'first'),  # Retain the first occurrence of EMPLOYER_NAME
    MEDICAL_ALLOWED_AMOUNT=('MEDICAL_ALLOWED_AMOUNT', 'sum'),
    MEDICAL_PAID_AMOUNT=('MEDICAL_PAID_AMOUNT', 'sum')
).reset_index()
med_agg

KeyError: "Column(s) ['EMPLOYER_NAME'] do not exist"

In [120]:
# Step 1: Aggregate by MEMBER_ID to get total RX paid amount per member
member_rx_agg = rx2.groupby('MEMBER_ID').agg(
    RX_ALLOWED_AMOUNT=('RX_ALLOWED_AMOUNT', 'sum'),
    RX_PAID_AMOUNT=('RX_PAID_AMOUNT', 'sum')
).reset_index()
member_rx_agg

,MEMBER_ID,RX_ALLOWED_AMOUNT,RX_PAID_AMOUNT
0,mma-0074b4b0405a4681a84a6a40fb58e4c4,5241.74,5056.74
1,mma-00cedc4e63a44cec83bd4661cc0d3978,674.24,609.24
2,mma-01ff2109e19f4bb892a5de6f123803a2,13297.33,13192.33
3,mma-021229d5337741629ee1d25ef74a9dd5,4850.08,4800.08
4,mma-02e37592ee22437da58f6a17400a74fe,150.00,5429.43
...,...,...,...
125,mma-f7eac770773440108081c9dee8b7cf99,9240.29,8762.82
126,mma-fba5559586804ff0a97ea991e474f0bb,165.64,165.64
127,mma-febd81d4d7ae4e73b8af7199139b8e67,3213.72,3113.72
128,mma-ff0ac00946ba4dd7a8ed9afe0f1d7258,3708.14,3608.14


In [121]:
member_agg = med_agg.merge(member_rx_agg, on='MEMBER_ID', how='left')
member_agg

,MEMBER_ID,EMPLOYER_NAME,MEDICAL_ALLOWED_AMOUNT,MEDICAL_PAID_AMOUNT,RX_ALLOWED_AMOUNT,RX_PAID_AMOUNT
0,mma-0074b4b0405a4681a84a6a40fb58e4c4,GRIFOLS,15084.44,13644.66,5241.74,5056.74
1,mma-00cedc4e63a44cec83bd4661cc0d3978,COLLIERS ENGINEERING AND DESIGN,2082.46,1907.46,674.24,609.24
2,mma-01ff2109e19f4bb892a5de6f123803a2,"CORPAY, INC.",12175.01,10883.43,13297.33,13192.33
3,mma-021229d5337741629ee1d25ef74a9dd5,TOKIO MARINE NORTH AMERICA,7828.07,7695.29,4850.08,4800.08
4,mma-02e37592ee22437da58f6a17400a74fe,PARKLAND HEALTH & HOSPITAL SYSTEM,6780.46,6780.46,150.00,5429.43
...,...,...,...,...,...,...
210,mma-f909d0d1bf85430594f027c933fa0a5e,PARKLAND HEALTH & HOSPITAL SYSTEM,7477.30,7477.30,NaN,NaN
211,mma-fba5559586804ff0a97ea991e474f0bb,LIVE OAK BANCSHARE,13475.93,13475.93,165.64,165.64
212,mma-febd81d4d7ae4e73b8af7199139b8e67,QUANTA SERVICES,2117.53,1404.17,3213.72,3113.72
213,mma-ff0ac00946ba4dd7a8ed9afe0f1d7258,QUANTA SERVICES,1963.71,1171.01,3708.14,3608.14


In [122]:
# Step 2: Group by SERVICE_PROVIDER_STATE to calculate sum and average of the aggregated medical paid amounts
employer_stats = member_agg.groupby('EMPLOYER_NAME').agg(
    Total_Medical_Allowed_Amount=('MEDICAL_ALLOWED_AMOUNT', 'sum'),
    Average_Medical_Allowed_Amount=('MEDICAL_ALLOWED_AMOUNT', 'mean'),
    Median_Medical_Allowed_Amount=('MEDICAL_ALLOWED_AMOUNT', 'median'),
    Total_Medical_Paid_Amount=('MEDICAL_PAID_AMOUNT', 'sum'),
    Average_Medical_Paid_Amount=('MEDICAL_PAID_AMOUNT', 'mean'),
    Median_Medical_Paid_Amount=('MEDICAL_PAID_AMOUNT', 'median'),
    Total_RX_Allowed_Amount=('RX_ALLOWED_AMOUNT', 'sum'),
    Average_RX_Allowed_Amount=('RX_ALLOWED_AMOUNT', 'mean'),
    Median_RX_Allowed_Amount=('RX_ALLOWED_AMOUNT', 'median'),
    Total_RX_Paid_Amount=('RX_PAID_AMOUNT', 'sum'),
    Average_RX_Paid_Amount=('RX_PAID_AMOUNT', 'mean'),
    Median_RX_Paid_Amount=('RX_PAID_AMOUNT', 'median'),
    Distinct_Member_Count=('MEMBER_ID','nunique')  # Count of distinct MEMBER_IDs
).reset_index()

# Sort columns
employer_stats.sort_values(by='Distinct_Member_Count', ascending=False)

,EMPLOYER_NAME,Total_Medical_Allowed_Amount,Average_Medical_Allowed_Amount,Median_Medical_Allowed_Amount,Total_Medical_Paid_Amount,Average_Medical_Paid_Amount,Median_Medical_Paid_Amount,Total_RX_Allowed_Amount,Average_RX_Allowed_Amount,Median_RX_Allowed_Amount,Total_RX_Paid_Amount,Average_RX_Paid_Amount,Median_RX_Paid_Amount,Distinct_Member_Count
41,PARKLAND HEALTH & HOSPITAL SYSTEM,274540.02,8856.13,8399.48,274540.02,8856.13,8399.48,1636.72,125.90,85.32,12862.32,989.41,234.79,31
44,QUANTA SERVICES,132053.32,6950.17,7244.99,106441.57,5602.19,5828.70,64233.51,3778.44,3490.28,54406.63,3200.39,3097.79,19
57,WAKEMED,158073.22,13172.77,11965.60,78306.62,6525.55,4668.78,24283.07,3469.01,4688.22,21331.64,3047.38,4142.98,12
30,INCYTE CORPORATION,157833.98,17537.11,14764.82,154237.88,17137.54,14162.35,89916.17,12845.17,1375.66,89661.17,12808.74,1340.66,9
54,VERISK ANALYTICS,83166.40,10395.80,8922.19,78911.72,9863.97,8432.49,32544.27,4649.18,3742.03,32134.27,4590.61,3712.03,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,HOOKER FURNITURE,3955.02,3955.02,3955.02,37.35,37.35,37.35,75.36,75.36,75.36,0.00,0.00,0.00,1
1,AMPLITY HEALTH,2618.54,2618.54,2618.54,1607.86,1607.86,1607.86,0.00,NaN,NaN,0.00,NaN,NaN,1
34,METROPOLIS,23176.38,23176.38,23176.38,22644.66,22644.66,22644.66,39.52,39.52,39.52,19.52,19.52,19.52,1
36,MISSOURI CHAMBER FEDERATION BENEFIT PLAN TRUST,4877.48,4877.48,4877.48,3383.14,3383.14,3383.14,0.00,NaN,NaN,0.00,NaN,NaN,1


In [123]:
pop = """
WITH MemberAges AS (
   SELECT 
        EMPLOYER_NAME,
        MEMBER_ID,
        DATEDIFF(YEAR, CONVERT(DATE, BIRTH_YEAR_MO + '-01'), GETDATE()) - 
        CASE 
            WHEN MONTH(CONVERT(DATE, BIRTH_YEAR_MO + '-01')) > MONTH(GETDATE()) 
                 OR (MONTH(CONVERT(DATE, BIRTH_YEAR_MO + '-01')) = MONTH(GETDATE()) 
                     AND DAY(CONVERT(DATE, BIRTH_YEAR_MO + '-01')) > DAY(GETDATE())) 
            THEN 1 
            ELSE 0 
        END AS AGE
    FROM 
        ACTUARIAL_AH.DBO.SN_MEMBER
    WHERE
        MONTH_KEY = '12-01-24'
        AND GENDER = 0
        AND MEDICAL_ENROLLMENT_STATUS = 3 
)

SELECT 
    MA.EMPLOYER_NAME,
    COUNT(DISTINCT MA.MEMBER_ID) AS TOTAL_MEMBERS
FROM 
    MemberAges MA
JOIN ACTUARIAL_AH.DBO.SN_MEDICAL M ON MA.MEMBER_ID = M.MEMBER_ID
WHERE 
    AGE BETWEEN 15 AND 49
    AND PAID_DATE >= '2024-01-01'
GROUP BY 
    MA.EMPLOYER_NAME
ORDER BY 
    MA.EMPLOYER_NAME
"""
pop = pd.read_sql(pop, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_52308\4020801657.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pop = pd.read_sql(pop, conn)


In [124]:
pop

,EMPLOYER_NAME,TOTAL_MEMBERS
0,ACCURIDE,231
1,ADMIRAL BEVERAGE CORPORATION,798
2,AESCULAPIAN MANAGEMENT COMPANY,212
3,AFNI INC,508
4,ALBEMARLE COUNTY & PUBLIC SCHOOLS,1856
...,...,...
292,"YAM MANAGEMENT, LLC",334
293,YEAR UP,460
294,ZENITH INSURANCE COMPANY,705
295,ZEP INC,257


In [125]:
pop_stats = employer_stats.merge(pop, on='EMPLOYER_NAME')
pop_stats

,EMPLOYER_NAME,Total_Medical_Allowed_Amount,Average_Medical_Allowed_Amount,Median_Medical_Allowed_Amount,Total_Medical_Paid_Amount,Average_Medical_Paid_Amount,Median_Medical_Paid_Amount,Total_RX_Allowed_Amount,Average_RX_Allowed_Amount,Median_RX_Allowed_Amount,Total_RX_Paid_Amount,Average_RX_Paid_Amount,Median_RX_Paid_Amount,Distinct_Member_Count,TOTAL_MEMBERS
0,ALPLA INC,17922.11,8961.06,8961.06,10885.10,5442.55,5442.55,0.00,NaN,NaN,0.00,NaN,NaN,2,392
1,AMPLITY HEALTH,2618.54,2618.54,2618.54,1607.86,1607.86,1607.86,0.00,NaN,NaN,0.00,NaN,NaN,1,300
2,BACARDI,19669.01,19669.01,19669.01,13563.76,13563.76,13563.76,0.00,NaN,NaN,0.00,NaN,NaN,1,723
3,BADGER METER INC,16621.02,16621.02,16621.02,12731.78,12731.78,12731.78,0.00,NaN,NaN,0.00,NaN,NaN,1,332
4,BANKERS HEALTHCARE,133776.43,26755.29,17000.00,115586.38,23117.28,14961.41,21098.69,10549.35,10549.35,17971.61,8985.81,8985.81,5,586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,WESTAT,8378.69,4189.35,4189.35,7688.64,3844.32,3844.32,9415.67,4707.84,4707.84,9115.67,4557.84,4557.84,2,431
60,WOODWARD ACADEMY,15230.32,15230.32,15230.32,13865.10,13865.10,13865.10,0.00,NaN,NaN,0.00,NaN,NaN,1,260
61,WORLDSTRIDES,1812.42,1812.42,1812.42,980.22,980.22,980.22,76.33,76.33,76.33,66.33,66.33,66.33,1,430
62,YEAR UP,22871.66,7623.89,7507.56,19365.56,6455.19,5629.48,9348.13,4674.07,4674.07,9198.13,4599.07,4599.07,3,460


In [126]:
pop_stats['PER_1K'] = (pop_stats['Distinct_Member_Count'] / pop_stats['TOTAL_MEMBERS']) * 1000
pop_stats.sort_values(by='PER_1K', ascending=False)

,EMPLOYER_NAME,Total_Medical_Allowed_Amount,Average_Medical_Allowed_Amount,Median_Medical_Allowed_Amount,Total_Medical_Paid_Amount,Average_Medical_Paid_Amount,Median_Medical_Paid_Amount,Total_RX_Allowed_Amount,Average_RX_Allowed_Amount,Median_RX_Allowed_Amount,Total_RX_Paid_Amount,Average_RX_Paid_Amount,Median_RX_Paid_Amount,Distinct_Member_Count,TOTAL_MEMBERS,PER_1K
38,NATIONAL GEOGRAPHIC,81349.88,20337.47,18569.64,77679.58,19419.90,17409.64,14881.28,14881.28,14881.28,14831.28,14831.28,14831.28,4,292,13.70
45,RICHLINE GROUP,22990.52,11495.26,11495.26,22750.52,11375.26,11375.26,3298.08,3298.08,3298.08,2998.08,2998.08,2998.08,2,157,12.74
21,FELICIAN UNIVERSITY,14023.52,14023.52,14023.52,13923.52,13923.52,13923.52,0.00,NaN,NaN,0.00,NaN,NaN,1,79,12.66
50,STOCKX,39180.44,13060.15,11880.81,35678.44,11892.81,10334.02,24312.29,8104.10,8926.63,23952.29,7984.10,8826.63,3,271,11.07
56,WAFRA,15147.17,15147.17,15147.17,11498.06,11498.06,11498.06,0.00,NaN,NaN,0.00,NaN,NaN,1,101,9.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,MCCORMICK,5815.73,5815.73,5815.73,4483.90,4483.90,4483.90,4677.54,4677.54,4677.54,4577.54,4577.54,4577.54,1,1815,0.55
48,SHI INTERNATIONAL CORP,8497.54,8497.54,8497.54,6111.81,6111.81,6111.81,0.00,NaN,NaN,0.00,NaN,NaN,1,2091,0.48
11,CHOP,-6178.04,-1544.51,8.17,-4868.27,-1217.07,-10.16,0.00,NaN,NaN,0.00,NaN,NaN,4,10528,0.38
36,MISSOURI CHAMBER FEDERATION BENEFIT PLAN TRUST,4877.48,4877.48,4877.48,3383.14,3383.14,3383.14,0.00,NaN,NaN,0.00,NaN,NaN,1,9676,0.10


In [127]:
ivf_stats_costs = pop_stats.to_csv('ivf_stats_costs.csv', index=False)